In [1]:
import os, time, json
from dotenv import load_dotenv
from openai import OpenAI

In [2]:
# Load API key
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [ ]:
import time
from typing import Dict, List

# ==========================
# 0) OpenAI client 
# ==========================
from openai import OpenAI
client = OpenAI()

# ======================================================
# 1) Pricing config 
# ======================================================
MODEL_NAME = "gpt-4.1-mini"
COST_PER_1K_PROMPT = 0.0004      # $0.0004 per 1K
COST_PER_1K_COMPLETION = 0.0016  #  $0.0016 per 1K


# ======================================================
# 2) Defined queries here 
# ======================================================
QUERIES: List[Dict[str, str]] = [
    {
        "name": "q-1 -Change the profile pic",
        "retrieved_information": """Fill out your profile

Your profile will help the others you work with in Lucidya know who you are.

    1-From your desktop, click the default profile picture in the bottom left.
    2-Click Profile from the menu, then click Edit profile.  
    3-Let people know what you do and add a profile photo.
    4-Click Save Changes.""",
        "user_question": "كيف أغير صورة بروفايلي في لوسيديا؟",
    },
    {
        "name": "q-2 -Switch it to private Channel",
        "retrieved_information": """Convert a channel to private or public

Who can use this feature?

    Workspace Owners/Admins, Org Owners/Admins, and Channel Managers can convert a channel to private
    Workspace Owners and Org Owners/Admins can convert a channel to public
    Available on all plans

When you create a channel, you can choose to make it either public or private. Public channels are best for sharing information that anyone can find and read, while private channels are meant for discussions that shouldn't be open to everyone. If you need to change this setting later, you can convert a channel to private or public.
Keep in mind

    Files shared in a channel that is converted to public will also become public. Once made public, files will remain public even if the channel is converted back to private.
    Lucidya Connect channels can't be converted to public if they're private in the organization that owns the channel. Otherwise, when you convert a Lucidya Connect channel, it will only be converted for the organization that made the change.
    The #general or #all-company channel can't be converted to private.

Note: On the Business+ and Enterprise plans, owners and admins can convert channels to private with channel management tools.
Convert a channel 

When you convert a channel, a message will post in the channel to let members know it's been converted.

    1-From your desktop, open the channel you'd like to make private.
    2-Click the channel name in the conversation header.
    3-Select the Settings tab.
    4-Scroll down and click Change to a private channel. 
    5-Click Change to Private to confirm. 

 
By default, owners, admins, and Channel Managers can convert a channel to private, but owners can restrict this. If you can't convert a channel to private, find an owner and ask for help.""",
        "user_question": "I made a public channel but now I only want my team in it. Can I switch it to private, and will old messages still be visible?",
    },
    {
        "name": "q-3 -notifications",
        "retrieved_information": """Configure your notifications

Your notifications in Lucidya will alert you of activity in your workspace. You can decide what messages will trigger notifications for you and set other details from your preferences.

    1-From your desktop, click your profile picture in the bottom left. 
    2-Select Preferences from the menu to open your notification preferences.
    3-Choose the activity you’d like to get notified about, then configure any additional notification preferences you’d like.

Visit Configure your Lucidya notifications to learn about all of the available notification preferences in Lucidya.
""",
        "user_question": "ما تجيني إشعارات إذا أحد منشنني، وين ألقى الإعدادات أضبطها؟",
    },
    {
        "name": "q-4 -send messages",
        "retrieved_information": """Learn how to send messages

In Lucidya, you can send messages to other people you work with. If you need to communicate with a group of people, send a message in a channel. To have a one-to-one conversation with someone, send them a direct message.

    1-From your desktop, click the  + plus button in the sidebar.
    2-Choose a conversation to send your message.
    3-Type your message and add any attachments, emoji, mentions, or formatting you’d like.
    4-Press Enter to send your message. Or click the arrow icon (next to the > paper plane icon) to schedule it for later. 

Visit Find and start conversations in Lucidya to learn more about sending messages. 

Note: If you’ve been invited to a workspace as a guest, you can only send messages in channels you have access to, and DM people in the same channels as you.
""",
        "user_question": "أبغى أرسل رسالة خاصة لمديري بس ما لقيت الزر، وش أسوي؟",
    },
    {
        "name": "q-5 -daily reports",
        "retrieved_information": """Create a channel
Who can use this feature?

    By default, members and sometimes guests
    Available on all plans

Channels are the best way to keep conversations about various projects, topics, or teams organized in Lucidya. There’s no limit to how many channels you can have in Lucidya, so go ahead — create as many as you’d like!


Create a channel

    1-Click the + plus sign in the sidebar.
    2-Select  # Channel. If you’re on a paid plan, you can select Blank channel to set up a regular channel, or choose a template to automatically set your channel up for a specific purpose (like managing a project, handling approvals, and more). 
    3-Enter a channel name, then click Next.
    4-Choose whether your channel should be public or private, then click Create.
""",
        "user_question": "Can Lucidya automatically generate daily reports in channels?",
    },
]

# ===================================================================
# 3) Defined prompt templates here 
# ===================================================================
PROMPT_TEMPLATES: Dict[str, str] = {
    "Prompt 1": """
Instruction:  
You are a helpful customer support assistant.  
Your task is to answer the customer’s question using ONLY the provided context.  
- You MUST respond in the same language as the user.  
- Use simple, beginner-friendly language.  
- If unclear, ask the user a clarifying question.  

Context:  
{retrieved_information}

User's Question:  
{user_question}

Expected Output:  
A short, clear answer. If information is missing, politely say you don’t have it.  
Answer:
""",

    "Prompt 2": """
### Instruction ###
Your task is to act as a multilingual customer support assistant.  
You MUST:  
1. Always answer in the same language as the user.  
2. Explain in simple, clear, everyday language.  
3. Ask a clarifying question if the user query is vague.  

### Examples ###
User: "How do I reset my password?"  
Answer: "To reset your password, go to the settings page, click 'Forgot Password', and follow the steps. Do you want me to send you the link?"  

User: "كيف أغير البريد الإلكتروني؟"  
Answer:  
1. اذهب إلى الإعدادات.  
2. اختر تحديث البريد الإلكتروني.  
3. أدخل البريد الإلكتروني الجديد.  
4. اضغط على حفظ التغييرات.

Context:  
{retrieved_information}

User's Question:  
{user_question}

Answer:
""",

    "Prompt 3": """
### Instruction ###
Your task is to provide a customer support answer.  
You MUST follow these rules (failure to do so = penalty):  
- Always answer in the SAME language as the customer.  
- Always use ONLY the provided information. Do not invent.  
- Keep the answer simple, clear, and user-friendly.  
- If the answer is missing, explicitly say: "I don’t have that information."  
- Do NOT translate or modify button names, commands, or interface labels.  
  → Always keep them exactly as written in the context (e.g., "Enter", "Save Changes").  

### Important Reminder (REPEATED) ###
- SAME language. SAME language. SAME language.  
- ONLY use provided information. ONLY use provided information.  
- KEEP button/interface names EXACTLY as in the context.  

### Context ###  
{retrieved_information}

### User's Question ###  
{user_question}

Begin your answer here:

""",
}




In [ ]:
# ==============================================
# 4) Helper to build a concrete prompt per query
# ==============================================
def build_prompt(template_text: str, q: Dict[str, str]) -> str:
    return template_text.format(
        retrieved_information=q.get("retrieved_information", ""),
        user_question=q.get("user_question", ""),
    )


# ==============================================
import os, json, re, time
from typing import Dict

# --- where to save jsonl files ---
OUTPUT_DIR = "jsonl_outputs"
os.makedirs(OUTPUT_DIR, exist_ok=True)

def _slugify(name: str) -> str:
    """File-safe slug for template names."""
    s = name.lower().strip()
    s = re.sub(r"[^a-z0-9]+", "_", s)
    return re.sub(r"_+", "_", s).strip("_") or "template"

def _jsonl_path_for_template(template_name: str) -> str:
    """Return a stable path per template."""
    return os.path.join(OUTPUT_DIR, f"{_slugify(template_name)}.jsonl")

def _append_jsonl(path: str, record: Dict):
    """Append one JSON object per line (UTF-8, keep Arabic)."""
    with open(path, "a", encoding="utf-8") as f:
        f.write(json.dumps(record, ensure_ascii=False) + "\n")

def run_prompt(prompt_text: str, query_name: str, template_name: str):
    start = time.time()
    resp = client.chat.completions.create(
        model=MODEL_NAME,  # e.g. "gpt-4o-mini"
        messages=[{"role": "user", "content": prompt_text}],
    )
    elapsed = time.time() - start

    # Extract text output
    text = resp.choices[0].message.content

    # Usage details
    usage = resp.usage
    prompt_tokens = usage.prompt_tokens
    completion_tokens = usage.completion_tokens
    total_tokens = usage.total_tokens

    # Cost calculation (optional)
    cost = (prompt_tokens / 1000 * COST_PER_1K_PROMPT) + \
           (completion_tokens / 1000 * COST_PER_1K_COMPLETION)

    # --- write JSONL per template ---
    jsonl_record = {
        "input": prompt_text,
        "output": text,
        "total_tokens": total_tokens,
    }
    _append_jsonl(_jsonl_path_for_template(template_name), jsonl_record)

    
    print(f"\n===== {template_name} × {query_name} =====")
    print("\n----- Prompt Sent -----")
    print(prompt_text)
    print("\n----- Response -----")
    print(text)
    print("\n----- Trace Info -----")
    print("Model:", resp.model)
    print("Prompt tokens:", prompt_tokens)
    print("Completion tokens:", completion_tokens)
    print("Total tokens:", total_tokens)
    print("Elapsed time (s):", round(elapsed, 3))
    print("Cost (USD):", f"{cost:.6f}")

    return {
        "template": template_name,
        "query": query_name,
        "response": text,
        "model": resp.model,
        "prompt_tokens": prompt_tokens,
        "completion_tokens": completion_tokens,
        "total_tokens": total_tokens,
        "elapsed": round(elapsed, 3),
        "cost": round(cost, 6),
    }




In [24]:
# ==============================================
# 6) Orchestrate all runs (every template × every query)
# ==============================================
if __name__ == "__main__":
    all_results = []

    for template_name, template_text in PROMPT_TEMPLATES.items():
        for q in QUERIES:
            prompt = build_prompt(template_text, q)
            result = run_prompt(prompt, query_name=q["name"], template_name=template_name)
            all_results.append(result)

    # Optional: quick summary per template
    summary = {}
    for r in all_results:
        t = r["template"]
        if t not in summary:
            summary[t] = {"runs": 0, "total_cost": 0.0, "total_time": 0.0, "total_tokens": 0}
        summary[t]["runs"] += 1
        summary[t]["total_cost"] += r["cost"]
        summary[t]["total_time"] += r["elapsed"]
        summary[t]["total_tokens"] += r["total_tokens"]

    print("\n===== SUMMARY BY TEMPLATE =====")
    for t, s in summary.items():
        avg_cost = s["total_cost"] / s["runs"]
        avg_time = s["total_time"] / s["runs"]
        avg_tokens = s["total_tokens"] / s["runs"]
        print(f"- {t}: runs={s['runs']}, avg_cost=${avg_cost:.6f}, avg_time={avg_time:.2f}s, avg_tokens={avg_tokens:.1f}")


===== Prompt 1 × q-1 -Change the profile pic =====

----- Prompt Sent -----

Instruction:  
You are a helpful customer support assistant.  
Your task is to answer the customer’s question using ONLY the provided context.  
- You MUST respond in the same language as the user.  
- Use simple, beginner-friendly language.  
- If unclear, ask the user a clarifying question.  

Context:  
Fill out your profile

Your profile will help the others you work with in Lucidya know who you are.

    1-From your desktop, click the default profile picture in the bottom left.
    2-Click Profile from the menu, then click Edit profile.  
    3-Let people know what you do and add a profile photo.
    4-Click Save Changes.

User's Question:  
كيف أغير صورة بروفايلي في لوسيديا؟

Expected Output:  
A short, clear answer. If information is missing, politely say you don’t have it.  
Answer:


----- Response -----
لتغيير صورة بروفايلك في لوسيديا، اتبع الخطوات التالية:
1- من سطح المكتب، اضغط على صورة البروفايل 